# 🤲 ทดสอบโมเดล Hand B - คู่มือสำหรับมือใหม่

> **สำหรับ Person 2: ผู้ดูแลโมเดล Hand B**

## 📖 เกี่ยวกับโมเดลนี้

โมเดลนี้จะจดจำท่าภาษามือไทย **9 คำ**:
- **ขอบคุณ** (มือแตะหน้าผากแล้วเหวี่ยงออก)
- **ไม่เป็นไร** (มือโบกซ้ายขวา)
- **สบายดี** (มือชูนิ้วโป้ง)
- **โชคดี** (มือกำปั้นชูขึ้น)
- **เก่ง** (นิ้วโป้งชูขึ้น)
- **อิ่ม** (มือถูท้องเป็นวงกลม)
- **หิว** (มือแตะท้อง)
- **เศร้า** (มือปิดหน้าหรือเช็ดน้ำตา)
- **idle** (มือพักธรรมชาติ)

---

## 🎯 วัตถุประสงค์

1. **ทดสอบโมเดลก่อนใช้งานจริง** - ตรวจสอบว่าโมเดลทำงานได้ดี
2. **วัดความแม่นยำ** - ดู confidence score ต้องสูงกว่า 60%
3. **ปรับปรุงโมเดล** - หาจุดอ่อนเพื่อกลับไป train ใหม่

---

## 📝 วิธีใช้งาน

1. **ขั้นตอนที่ 1:** รัน Cell 1 เพื่อติดตั้ง libraries (ครั้งแรกเท่านั้น)
2. **ขั้นตอนที่ 2:** รัน Cell 2 เพื่อโหลดโมเดลและสร้าง testing interface
3. **ขั้นตอนที่ 3:** รัน Cell 3 เพื่อทดสอบด้วยรูปภาพ
4. **ขั้นตอนที่ 4:** รัน Cell 4 เพื่อทดสอบด้วยกล้องแบบ Real-time

⚠️ **ข้อควรจำ:** รันเซลล์ทั้งหมดตามลำดับ อย่าข้าม!

In [ ]:
# 📦 ขั้นตอนที่ 1: ติดตั้ง Libraries และโหลดโมเดล

print("🔄 กำลังติดตั้ง libraries...")
print("⏰ กรุณารอสักครู่ (อาจใช้เวลา 1-2 นาที)")

import sys
!{sys.executable} -m pip install tensorflow opencv-python pillow numpy matplotlib

# Import libraries ทั้งหมด
print("\n📖 กำลัง import libraries...")
import tensorflow as tf
import json
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import os

# แสดงข้อมูลระบบ
print("="*60)
print("🖥️  ข้อมูลระบบ:")
print(f"   TensorFlow Version: {tf.__version__}")
print(f"   📁 Current Directory: {os.getcwd()}")

# ตรวจสอบไฟล์โมเดลที่จำเป็น
print("\n🔍 ตรวจสอบไฟล์โมเดล...")
required_files = ['model.json', 'metadata.json']
missing_files = []
found_files = []

for file in required_files:
    if os.path.exists(file):
        found_files.append(file)
        print(f"   ✅ พบไฟล์: {file}")
    else:
        missing_files.append(file)
        print(f"   ❌ ไม่พบไฟล์: {file}")

print("="*60)

class HandBModel:
    """
    โคลาสสำหรับจัดการโมเดล Teachable Machine Hand B
    งานหลัก: โหลดโมเดล, เตรียมรูปภาพ, ทำนายผล
    """
    
    def __init__(self):
        print("🔧 สร้าง HandB Model instance...")
        self.model = None
        self.class_names = []
        self.input_shape = (224, 224, 3)
        print("✅ Instance พร้อมแล้ว!")
        
    def load_model(self, model_path='.'):
        """โหลดโมเดลจากไฟล์"""
        print("📂 กำลังโหลดโมเดล...")
        
        try:
            # โหลด metadata
            print("   📄 อ่านไฟล์ metadata.json...")
            with open(os.path.join(model_path, 'metadata.json'), 'r', encoding='utf-8') as f:
                metadata = json.load(f)
                self.class_names = [item['className'] for item in metadata['labels']]
                
            # โหลดโมเดล TensorFlow.js
            print("   🧠 โหลดโมเดล AI...")
            model_url = os.path.join(model_path, 'model.json')
            self.model = tf.keras.models.load_model(model_url)
            
            # แสดงผลสำเร็จ
            print("="*60)
            print("🎉 โหลดโมเดลสำเร็จ!")
            print(f"📊 จำนวนคลาสทั้งหมด: {len(self.class_names)}")
            print("📝 รายชื่อคลาส:")
            for i, name in enumerate(self.class_names, 1):
                print(f"   {i:2d}. {name}")
            print("="*60)
            
        except Exception as e:
            print("="*60)
            print("❌ เกิดข้อผิดพลาดขณะโหลดโมเดล!")
            print(f"🐛 ข้อผิดพลาด: {e}")
            print("\n💡 วิธีแก้ไข:")
            print("   1. ตรวจสอบว่าไฟล์ model.json และ metadata.json อยู่ในโฟลเดอร์")
            print("   2. ตรวจสอบว่าไฟล์ weights.bin ครบถ้วน")
            print("   3. ลองดาวน์โหลดโมเดลใหม่จาก Teachable Machine")
            print("="*60)
    
    def preprocess_image(self, image_path_or_array):
        """เตรียมรูปภาพให้พร้อมสำหรับการทำนาย"""
        try:
            # อ่านรูปภาพ
            if isinstance(image_path_or_array, str):
                image = Image.open(image_path_or_array).convert('RGB')
            else:
                image = Image.fromarray(image_path_or_array).convert('RGB')
            
            # ปรับขนาดเป็น 224x224
            image = image.resize((224, 224))
            image_array = np.array(image) / 255.0
            image_array = np.expand_dims(image_array, axis=0)
            return image_array
            
        except Exception as e:
            print(f"❌ ไม่สามารถเตรียมรูปภาพได้: {e}")
            return None
    
    def predict(self, image_path_or_array, show_details=True):
        """ทำนายผลจากรูปภาพ"""
        if self.model is None:
            print("❌ โปรดโหลดโมเดลก่อน (รัน cell ด้านบน)")
            return None
            
        processed_image = self.preprocess_image(image_path_or_array)
        if processed_image is None:
            return None
        
        predictions = self.model.predict(processed_image, verbose=0)[0]
        max_index = np.argmax(predictions)
        max_confidence = predictions[max_index]
        predicted_class = self.class_names[max_index]
        
        result = {
            'predicted_class': predicted_class,
            'confidence': float(max_confidence),
            'all_predictions': {name: float(conf) for name, conf in zip(self.class_names, predictions)}
        }
        
        if show_details:
            print("="*50)
            print("🎯 ผลการทำนาย:")
            print(f"   🏆 คลาสที่ทำนาย: {predicted_class}")
            print(f"   📊 ความแม่นยำ: {max_confidence:.1%}")
            
            if max_confidence >= 0.8:
                print("   ⭐ ความแม่นยำสูงมาก! โมเดลมั่นใจมาก")
            elif max_confidence >= 0.6:
                print("   ✅ ความแม่นยำดี โมเดลทำนายถูกต้อง")
            else:
                print("   ⚠️  ความแม่นยำต่ำ ควรถ่ายรูปใหม่หรือปรับปรุงโมเดล")
            
            print("\n📈 ความแม่นยำทั้งหมด:")
            for name, conf in result['all_predictions'].items():
                bar = "█" * int(conf * 20)
                print(f"   {name:12s} {conf:.1%} {bar}")
            print("="*50)
        
        return result

# สร้างและโหลดโมเดล
print("🚀 กำลังสร้างโมเดล Hand B...")
handB_model = HandBModel()

if not missing_files:
    print("\n📂 กำลังโหลดโมเดลจากไฟล์...")
    handB_model.load_model('.')
else:
    print(f"\n⚠️  ไฟล์ที่ขาดหาย: {missing_files}")
    print("💡 โปรดวางไฟล์โมเดลในโฟลเดอร์นี้แล้วรันใหม่")

In [ ]:
# 🖼️ ขั้นตอนที่ 2: ทดสอบด้วยรูปภาพ

def test_with_image_file(image_filename="test_image.jpg"):
    """
    ทดสอบโมเดล Hand B ด้วยไฟล์รูปภาพ
    
    วิธีใช้:
    1. วางไฟล์รูปภาพในโฟลเดอร์เดียวกับ notebook นี้
    2. เปลี่ยนชื่อไฟล์ใน function นี้
    3. รัน cell
    """
    
    print("🔍 กำลังค้นหาไฟล์รูปภาพ...")
    print(f"📄 ชื่อไฟล์: {image_filename}")
    
    if os.path.exists(image_filename):
        print(f"✅ พบไฟล์ {image_filename}")
        
        # แสดงรูปภาพ
        print("\n🖼️  แสดงรูปภาพที่ใช้ทดสอบ:")
        img = Image.open(image_filename)
        
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"📸 ภาพทดสอบ Hand B: {image_filename}", fontsize=14, pad=20)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        # ทำนายผล
        print("\n🤖 กำลังทำนายผล...")
        result = handB_model.predict(image_filename, show_details=True)
        
        if result:
            confidence = result['confidence']
            prediction = result['predicted_class']
            
            print("\n🎉 สรุปผลการทดสอบ Hand B:")
            if confidence >= 0.8:
                print(f"   🌟 เยี่ยมมาก! ระบบทำนายได้: '{prediction}' ด้วยความแม่นยำ {confidence:.1%}")
                print("   💡 โมเดลทำงานได้ดีมาก พร้อมใช้งานจริง")
            elif confidence >= 0.6:
                print(f"   ✅ ดี! ระบบทำนายได้: '{prediction}' ด้วยความแม่นยำ {confidence:.1%}")
                print("   💡 โมเดลทำงานได้ดี สามารถใช้งานได้")
            else:
                print(f"   ⚠️  ระบบทำนาย: '{prediction}' แต่ความแม่นยำต่ำ ({confidence:.1%})")
                print("   💡 คำแนะนำ:")
                print("      - ลองถ่ายรูปใหม่ในแสงที่ดีกว่า")
                print("      - ทำท่าให้ชัดเจนมากขึ้น")
                print("      - อาจต้องเพิ่มข้อมูลการ train")
            
            # แสดงกราฟ confidence
            plt.figure(figsize=(12, 6))
            names = list(result['all_predictions'].keys())
            scores = list(result['all_predictions'].values())
            
            colors = ['#ff6b6b' if score == max(scores) else '#4ecdc4' for score in scores]
            
            plt.barh(names, scores, color=colors)
            plt.xlabel('Confidence Score')
            plt.title(f'Hand B Model - Confidence Scores for: {image_filename}', fontsize=14, pad=20)
            plt.xlim(0, 1)
            
            # เพิ่มเส้นแสดงเกณฑ์
            plt.axvline(x=0.6, color='orange', linestyle='--', alpha=0.7, label='Min Threshold (60%)')
            plt.axvline(x=0.8, color='green', linestyle='--', alpha=0.7, label='Good Threshold (80%)')
            
            for i, score in enumerate(scores):
                plt.text(score + 0.01, i, f'{score:.1%}', va='center')
            
            plt.legend()
            plt.tight_layout()
            plt.show()
                
    else:
        print(f"❌ ไม่พบไฟล์ {image_filename}")
        print("\n💡 วิธีเพิ่มรูปทดสอบ:")
        print("   1. ถ่ายรูปท่าภาษามือที่ต้องการทดสอบ")
        print("   2. บันทึกรูปในโฟลเดอร์เดียวกับ notebook นี้")
        print("   3. เปลี่ยนชื่อไฟล์ในฟังก์ชันนี้")
        print(f"   4. ไฟล์ที่รองรับ: .jpg, .jpeg, .png")

# ทดสอบทันที
print("🚀 เริ่มทดสอบโมเดล Hand B ด้วยรูปภาพ...")
test_with_image_file("test_image.jpg")

print("\n" + "="*60)
print("💡 เคล็ดลับการทดสอบ Hand B:")
print("   📸 ถ่ายรูปท่าภาษามือที่ชัดเจน")
print("   💡 ทดสอบในแสงที่ดี")
print("   🔄 ลองทุกท่าใน Hand B: ขอบคุณ, ไม่เป็นไร, สบายดี, โชคดี, เก่ง, อิ่ม, หิว, เศร้า, idle")
print("   📊 confidence > 60% = ผ่าน")
print("="*60)

In [ ]:
# 📹 ขั้นตอนที่ 3: ทดสอบด้วยกล้องเว็บแคมแบบ Real-time

def run_webcam_test():
    """
    ทดสอบโมเดล Hand B ด้วยกล้องเว็บแคมแบบ Real-time
    
    คำแนะนำ:
    - กด 'S' เพื่อถ่ายรูปและทำนาย
    - กด 'Q' เพื่อออกจากระบบ
    - ทำท่าให้ชัดเจน หันหน้าเข้ากล้อง
    - ทดสอบทุกท่าใน Hand B
    """
    
    print("📹 กำลังเปิดกล้องเว็บแคม สำหรับ Hand B...")
    
    try:
        # เปิดกล้อง
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("❌ ไม่สามารถเปิดกล้องได้!")
            print("💡 วิธีแก้ไข:")
            print("   - ตรวจสอบว่ากล้องเชื่อมต่ออยู่")
            print("   - ปิดโปรแกรมอื่นที่ใช้กล้อง")
            print("   - รีสตาร์ท Jupyter Notebook")
            return
        
        print("✅ กล้องพร้อมใช้งาน!")
        print("\n" + "="*60)
        print("🎮 การควบคุม:")
        print("   📸 กด 'S' = ถ่ายรูปและทำนาย")
        print("   🚪 กด 'Q' = ออกจากระบบ")
        print("\n🎯 ท่าที่ต้องทดสอบ (Hand B):")
        print("   1. ขอบคุณ    2. ไม่เป็นไร   3. สบายดี")
        print("   4. โชคดี     5. เก่ง        6. อิ่ม")
        print("   7. หิว       8. เศร้า       9. idle")
        print("="*60)
        
        capture_count = 0
        gesture_log = []  # เก็บบันทึกการทดสอบ
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ ไม่สามารถอ่านภาพจากกล้องได้")
                break
            
            # แสดงข้อความบนหน้าจอ
            cv2.putText(frame, "Hand B Model Test", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, "Press 'S' to capture, 'Q' to quit", 
                       (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"Tests: {capture_count}", 
                       (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
            # แสดงกรอบสีฟ้าเพื่อให้รู้ว่าเป็น Hand B
            cv2.rectangle(frame, (50, 50), (590, 430), (255, 100, 0), 2)
            cv2.putText(frame, "Hand B Test Zone", 
                       (60, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
            
            cv2.imshow('HandB Model - Webcam Test', frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('s') or key == ord('S'):  # ถ่ายรูป
                capture_count += 1
                print(f"\n📸 การทดสอบ Hand B ครั้งที่ {capture_count}")
                
                # แปลง BGR to RGB
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # ทำนายผล
                print("🤖 กำลังทำนาย Hand B...")
                result = handB_model.predict(rgb_frame, show_details=False)
                
                if result:
                    prediction = result['predicted_class']
                    confidence = result['confidence']
                    
                    # บันทึกผล
                    gesture_log.append({
                        'test': capture_count,
                        'prediction': prediction,
                        'confidence': confidence
                    })
                    
                    # แสดงผลใน terminal
                    print(f"🎯 ผล Hand B: {prediction} | ความแม่นยำ: {confidence:.1%}")
                    
                    # แสดงผลบนภาพ
                    result_text = f"HandB: {prediction} ({confidence:.1%})"
                    cv2.putText(frame, result_text, 
                               (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 100, 255), 2)
                    
                    # ประเมินผล
                    if confidence >= 0.8:
                        status_text = "Excellent!"
                        color = (0, 255, 0)  # เขียว
                        print("   ⭐ ทำนายได้เยี่ยม!")
                    elif confidence >= 0.6:
                        status_text = "Good!"
                        color = (0, 255, 255)  # เหลือง
                        print("   ✅ ทำนายได้ดี!")
                    else:
                        status_text = "Try Again"
                        color = (0, 100, 255)  # ส้ม
                        print("   ⚠️  ความแม่นยำต่ำ ลองทำท่าใหม่")
                    
                    cv2.putText(frame, status_text, 
                               (10, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    
                    # แสดงคำแนะนำท่าถัดไป
                    suggestions = ["ขอบคุณ", "ไม่เป็นไร", "สบายดี", "โชคดี", "เก่ง", "อิ่ม", "หิว", "เศร้า", "idle"]
                    if capture_count <= len(suggestions):
                        next_gesture = suggestions[capture_count % len(suggestions)]
                        print(f"   💡 ลองท่าถัดไป: {next_gesture}")
                        cv2.putText(frame, f"Try: {next_gesture}", 
                                   (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    # แสดงผลเป็นเวลา 3 วินาที
                    cv2.imshow('HandB Model - Result', frame)
                    cv2.waitKey(3000)
                
            elif key == ord('q') or key == ord('Q'):  # ออกจากโปรแกรม
                print("\n? ปิดกล้อง...")
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
        # สรุปผลการทดสอบ
        print("\n" + "="*60)
        print("📊 สรุปผลการทดสอบ Hand B:")
        print(f"   🔢 จำนวนการทดสอบทั้งหมด: {capture_count} ครั้ง")
        
        if gesture_log:
            high_conf = [g for g in gesture_log if g['confidence'] >= 0.8]
            good_conf = [g for g in gesture_log if g['confidence'] >= 0.6]
            
            print(f"   ⭐ ความแม่นยำสูง (≥80%): {len(high_conf)} ครั้ง")
            print(f"   ✅ ความแม่นยำดี (≥60%): {len(good_conf)} ครั้ง")
            print(f"   📈 อัตราความสำเร็จ: {len(good_conf)/len(gesture_log)*100:.1f}%")
            
            print("\n? รายละเอียดการทดสอบ:")
            for g in gesture_log:
                status = "⭐" if g['confidence'] >= 0.8 else "✅" if g['confidence'] >= 0.6 else "⚠️"
                print(f"   {status} ครั้งที่ {g['test']:2d}: {g['prediction']:10s} ({g['confidence']:.1%})")
        
        print("="*60)
        print("✅ เสร็จสิ้นการทดสอบ Hand B")
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดกับกล้อง: {e}")
        print("💡 วิธีแก้ไข:")
        print("   1. รีสตาร์ท Jupyter Notebook")
        print("   2. ตรวจสอบว่ากล้องทำงานปกติ")
        print("   3. ปิดโปรแกรมอื่นที่ใช้กล้อง")

print("🎥 ฟังก์ชันทดสอบกล้อง Hand B พร้อมใช้งาน")
print("\n💡 วิธีเริ่มทดสอบ:")
print("   รันคำสั่ง: run_webcam_test()")

print("\n🔥 คำแนะนำสำหรับการทดสอบ Hand B:")
print("   1. ทำท่าภาษามือให้ชัดเจนตามรายการ 9 ท่า")
print("   2. อยู่ในแสงที่ดี")  
print("   3. มือให้อยู่ในกรอบสีฟ้า")
print("   4. ทดสอบครบทุกท่าเพื่อวัดความแม่นยำโดยรวม")
print("   5. บันทึกผลเพื่อประเมินการปรับปรุงโมเดล")

print("\n" + "="*60)
print("🚀 พิมพ์คำสั่งนี้เพื่อเริ่มทดสอบ Hand B:")
print("run_webcam_test()")
print("="*60)

## 🎯 สรุปและขั้นตอนถัดไป

### ✅ สิ่งที่คุณควรได้จากการทดสอบ:

1. **ความแม่นยำโดยรวม ≥ 60%** - โมเดลพร้อมใช้งาน
2. **แยกแยะ idle ได้ดี** - ระบบจะไม่ทำนายผิดเมื่อไม่ได้ทำท่า
3. **ทุกท่าใน Hand B ทำนายถูก** - ครบ 9 ท่า

### 🔄 หากผลการทดสอบไม่ดี:

**ปัญหา: ความแม่นยำต่ำ (< 60%)**
- 📸 กลับไปเพิ่มรูปในคลาสที่มีปัญหา
- 💡 ถ่ายรูปในสภาพแสงและมุมมองที่หลากหลาย
- 🔄 Train โมเดลใหม่ด้วย Epochs สูงขึ้น (150-200)

**ปัญหา: สับสนระหว่างคำ**
- 🎯 เพิ่มรูปเพื่อแยกแยะคำที่คล้ายกัน
- 📊 ตรวจสอบว่าท่าต่างกันชัดเจนหรือไม่

**ปัญหา: idle ไม่แม่นยำ**
- 📈 เพิ่มรูป idle ให้มากขึ้น (150-200 รูป)
- 🌟 ถ่าย idle หลากหลายท่าวาง

---

### 🚀 ขั้นตอนนำไปใช้ใน Frontend:

1. **คัดลอกไฟล์โมเดล** ไปยัง `public/models/handB/`
2. **ทดสอบใน Frontend** ด้วยคำสั่ง `npm run dev`
3. **ประสานกับทีม** Person 3 (Frontend) และ Person 4 (Backend)
4. **ทดสอบระบบรวม** ร่วมกับโมเดล Hand A

---

### 📞 ติดต่อทีมเมื่อ:

- **Person 1:** เปรียบเทียบผลกับ Hand A
- **Person 3:** ปรับแต่งการแสดงผลใน Frontend  
- **Person 4:** ตั้งค่าการส่งข้อมูลไป Backend
- **ทีม:** รายงานผลการทดสอบโดยรวม

---

### 🎉 เมื่อโมเดลพร้อมแล้ว:

✅ **ความแม่นยำผ่านเกณฑ์**  
✅ **ทดสอบครบทุกท่า**  
✅ **ไฟล์โมเดลพร้อมใช้**  
✅ **ส่งมอบให้ทีม Frontend**  

**🎊 ยินดีด้วย! โมเดล Hand B ของคุณพร้อมใช้งานแล้ว**